# First Aid Kit

It's a pity, but it is impossible to develop applications that work perfectly.
Applications always contain errors due to programming mistakes, incorrect use of function calls, or external reasons, such as a missing file or the abnormal termination of an external (non-APL) piece of code.

All these problems may interrupt the normal processing of an application, and require immediate corrective action.
The aim of this chapter is to give you the basis to deal with these situations, and work through the experience.

In [a later chapter](./Event-Handling.ipynb) you will see that it is possible to trap and hide unpredictable errors or events, in order to provide a safe environment for the user.

Because APL is an interpreted language, program execution is not aborted, just suspended when an exception occurs.
Then the programmer has access to all the application variables, and can diagnose the problem.
If they can modify the code or correct some environmental dependency, they may be able to restart the program from the very point it had been interrupted in, until a new exception occurs, and so on.
This gives great flexibility when debugging applications.

In this chapter you will learn how to:
 - interpret error messages;
 - interpret the information provided by various system indicators;
 - trace the execution of an application step by step; and
 - set *breakpoints* in strategic places to help debugging.
 
A forgotten comma, an incorrect axis, a missing parenthesis, a scalar where you expected a vector, can all cause an error and it may sometimes be very difficult for a beginner to understand the reason.
Try to get help from people who have a better understanding of APL:
you will save time *and* learn a lot from them.

## When an Error Occurs

### Our First Error

Even when you use a function, that is in itself perfectly correct, errors may occur if you call it with arguments that are not consistent with the function's requirements.
We will use this type of mistake to illustrate what happens when an error occurs.

In the previous chapter we wrote a function to interlace two matrices of the same shape:

In [6]:
∇ r ← x Interlace y; size; even
    size ← 1 2×⍴x
    r ← size⍴0
    even ← 2×⍳(⍴x)[2]
    r[;even] ← y
    r[;even-1] ← x
∇

In [7]:
(2 2⍴1) (2 2⍴0)

┌───┬───┐
│1 1│0 0│
│1 1│0 0│
└───┴───┘

In [8]:
(2 2⍴1) Interlace (2 2⍴0)

1 0 1 0
1 0 1 0

This function works perfectly if you stick to its rules, but what happens if you call it with matrices that *don't* have the same shape?

#### Your Environment and Indicators

Let us try to interlace our `forecast` matrix, that has 4 rows and 6 columns,

In [10]:
⎕RL ← 73
⎕← forecast ← 10×?4 6⍴55

90 160 420 500 20 30
110 450 170 370 290 360
340 190 320 120 510 370
150 460 240 520 490 280

with a 3 by 5 matrix, and see what happens:

In [5]:
forecast Interlace 3 5⍴⍳15

LENGTH ERROR
Interlace[4] r[;even]←y
              ∧


The first thing we see is the error message that is issued.
Then, if you are using RIDE or the Windows interpreter, you will see that, in the lower right corner of the window, some information is highlighted.
<!--figure-->The first figure below<!--Win_Red_State_Indicator--> shows this highlighting in red, for users of the Windows interpreter, and <!--figure-->the next figure below<!--RIDE_Purple_State_Indicator--> shows this highlighting in purple for users of the RIDE interpreter.

![Red highlighting of `⎕SI` upon reaching an error in the Windows interpreter.](res/Win_Red_State_Indicator.png)

<br>

![Purple highlighting of `⎕SI` upon reaching an error in the RIDE interpreter.](res/RIDE_Purple_State_Indicator.png)

Furthermore, when using the Windows or RIDE interpreters, an additional trace windows appears, generally at the bottom, as shown respectively by <!--figure-->the next figures<!--Win_Trace_Window,RIDE_Trace_Window-->.

![Trace window in the Windows interpreter.](res/Win_Trace_Window.png)

---

![Trace window in the RIDE interpreter.](res/RIDE_Trace_Window.png)

APL will preserve the entire execution context (local variables and program status indicators), so that you can resume function execution once the error has been diagnosed and repaired.

Let us now examine the meaning of these things.

#### The Diagnostic Message

The *diagnostic message* is displayed on three rows of the screen which show:
 - the type of error (explained later in [this section](#Execution-Errors));
 - the function name [and line number] - the statement that can't be executed; and
 - a caret (the character `∧`) which is placed where the statement was interrupted.
 
A *system function* named `⎕DM` returns the latest *diagnostic message*.
It is a three-item nested vector, with a sub-vector for each of the three lines, as shown below:

In [10]:
)copy display DISPLAY
DISPLAY ⎕DM

C:\Program Files\Dyalog\Dyalog APL-64 18.0 Unicode\ws\display.dws saved Tue Dec 1 20:08:12 2020

┌→───────────────────────────────────────────────────────────┐
│ ┌→───────────┐ ┌→──────────────────────┐ ┌→──────────────┐ │
│ │LENGTH ERROR│ │Interlace[4] r[;even]←y│ │ ∧│ │
│ └────────────┘ └───────────────────────┘ └───────────────┘ │
└∊───────────────────────────────────────────────────────────┘

`⎕DM` retains this value until it is replaced by the occurrence of a new error, or until we explicitly clear it.

#### State Indicator and Line Counter

Whenever function execution is interrupted for any reason, Dyalog APL keeps track of the exact point where the interrupt occurred in the *state indicator* (SI for short).
This value of the state indicator may be obtained using a system command `)si` or a system function `⎕SI`:

 - system command:

```APL
      )si
#.Interlace[4]*
```

 - system function:

```APL
      ⎕SI
┌─────────┐
│Interlace│
└─────────┘
```

The *system function* `⎕SI` returns a nested vector containing only the names of the interrupted functions;
in this example only one name appears.

The *system command* `)si` reports, for each function in the execution stack:
 - a path (here `#.`) which will be explained in [a later chapter](./Namespaces.ipynb). Ignore it for now;
 - the function name (`Interlace`);
 - the number of the statement on which function execution has stopped (`[4]` in this case); and
 - an asterisk which means that this function has been interrupted.
 
What you see highlighted in the bottom right-hand corner of your session (`⎕SI:1`) is the number of functions referenced in the *state indicator*.
Normally, this number should be zero and displayed in black; as soon as an error or an interrupt occurs, it is displayed in a different colour as a warning (in red or purple, depending on your software).

APL also keeps, in a *line counter*, the list of function lines waiting for execution.
In this case there is only one.
It can be obtained from the *system function* `⎕LC`:

```APL
      ⎕LC
4
```

The number above shows that `Interlace` was interrupted on line 4.

<!-- begin note -->
***Note***:

 > Due to limitations of the Jupyter interface, many of the things we will be discussing here will work differently when applied directly to a Dyalog APL notebook.
<!-- end -->

For example, even though we got an error with the incorrect usage of `Interlace` and we were able to check the diagnostic message with `⎕DM`, the *state indicator* and the *line counter* are empty:

In [11]:
forecast Interlace 3 5⍴⍳15

LENGTH ERROR
Interlace[4] r[;even]←y
              ∧


In [12]:
⎕SI

In [13]:
⎕LC

#### The Trace Window

If you are running Dyalog APL using the default *tracer* behaviour, when you reach an error inside a function a window should pop up in the bottom of the interpreter, with the function text displayed in another colour and with the statement in error highlighted.

If you are using the Windows interpreter and the trace window did not appear spontaneously, just press <kbd>Ctr</kbd>+<kbd>Enter</kbd>. Then, go to "Options" &#8680; "Configuration" &#8680; "Trace/Edit" and tick the checkbox next to "*Show trace stack on error*".

We shall see later how this trace window can be used.
**Do not close it** for the moment.

#### You Can See the Local Variables

Your cursor may not be in this (trace) window.